In [1]:
# Setup path to import from src directory
import sys
import os
from pathlib import Path

# Get the current notebook's directory (tests/)
notebook_dir = Path.cwd()
print(f"Current directory: {notebook_dir}")

# Go up one level to get project root
project_root = notebook_dir.parent
print(f"Project root: {project_root}")

# Add src to Python path
src_path = project_root
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))
    
print(f"Added to path: {src_path}")
print(f"sys.path now includes: {sys.path[0]}")


Current directory: /Users/ernest/Desktop/Developer/langchain-bd/tests
Project root: /Users/ernest/Desktop/Developer/langchain-bd
Added to path: /Users/ernest/Desktop/Developer/langchain-bd
sys.path now includes: /opt/homebrew/Cellar/python@3.13/3.13.5/Frameworks/Python.framework/Versions/3.13/lib/python313.zip


In [2]:
# Now you can import from src
from src.tools import blockchain_tools

print(f"Available tools: {[attr for attr in dir(blockchain_tools) if 'tool' in attr]}")

Available tools: ['available_blockchains_tool', 'available_timeframes_tool', 'categories_by_gas_fees_tool', 'get_latest_growthepie_datasets_tool', 'tool', 'top_contracts_by_gas_fees_tool']


In [ ]:
def categories_by_gas_fees_tool(blockchain_name: str, timeframe: str = "7d") -> dict:
    """
    Gets categories by gas fees share data for specified blockchain networks from GrowThePie.
    Returns a dict with summary and data or error.
    """
    try:
        json_path = "src/data/inspect_blockspace.json"
        available_blockchains = get_available_blockchains(json_file_path=json_path)
        if blockchain_name.lower() not in [bc.lower() for bc in available_blockchains]:
            return {"error": f"Blockchain '{blockchain_name}' not supported.", "available_blockchains": available_blockchains}
        valid_timeframes = ["1d", "7d", "30d"]
        if timeframe not in valid_timeframes:
            return {"error": f"Timeframe '{timeframe}' not supported.", "valid_timeframes": valid_timeframes}
        df = get_categories_by_gas_fees_share(blockchain_name.lower(), timeframe, json_file_path=json_path)
        categories = df.to_dict(orient="records")
        # Compute summary fields
        if not categories:
            return {"error": "No category data found."}
        # Sort by gas_fees_share_usd descending
        sorted_cats = sorted(categories, key=lambda x: x.get("gas_fees_share_usd", 0), reverse=True)
        top_category = sorted_cats[0]["category"]
        top_category_share = sorted_cats[0]["gas_fees_share_usd"]
        category_breakdown = {cat["category"]: cat["gas_fees_share_usd"] for cat in categories}
        total_gas_fees_usd = sum(cat.get("gas_fees_usd_absolute", 0) for cat in categories)
        category_concentration = sum(cat.get("gas_fees_share_usd", 0) for cat in sorted_cats[:3])
        return {
            "blockchain": blockchain_name,
            "timeframe": timeframe,
            "top_category": top_category,
            "top_category_share": top_category_share,
            "category_breakdown": category_breakdown,
            "total_gas_fees_usd": total_gas_fees_usd,
            "category_concentration": category_concentration,
            "categories": categories,
            "columns": list(df.columns),
            "error": None
        }
    except Exception as e:
        return {"error": str(e)}

In [3]:
blockchain_tools.categories_by_gas_fees_tool.invoke({"blockchain_name": "mantle", "timeframe": "7d"})

{'error': "[Errno 2] No such file or directory: 'src/data/inspect_blockspace.json'"}

In [ ]:
def test_categories_by_gas_fees_tool():
    result = blockchain_tools.categories_by_gas_fees_tool.invoke({"blockchain_name": "mantle", "timeframe": "7d"})
    print("categories_by_gas_fees_tool result:", result)
    assert isinstance(result, dict)
    assert result["error"] is None
    assert "categories" in result
    assert isinstance(result["categories"], list)
    print("test_categories_by_gas_fees_tool passed.")

In [ ]:
test_categories_by_gas_fees_tool()